In [3]:
import torch
from torch.nn import MSELoss

In [5]:
t = torch.rand((1, 40, 18))

In [7]:
t = t.squeeze()
t.shape

torch.Size([40, 18])

In [10]:
a1 = t[:, [0,1]]
a2 = t[:, [6,7]]
b  = t[:, [12,13]]

In [11]:
a1.shape, a2.shape, b.shape

(torch.Size([40, 2]), torch.Size([40, 2]), torch.Size([40, 2]))

In [13]:
from torch.nn import PairwiseDistance
dist = PairwiseDistance(p=2)

In [18]:
dis_a1_a2 = dist(a1, a2)
dis_a1_b = dist(a1, b)
dis_b_a2 = dist(b, a2)
dis_a1_a2.shape

torch.Size([40])

In [21]:
distances = torch.stack([dis_a1_a2, dis_a1_b, dis_b_a2], dim = 1)
distances.shape

torch.Size([40, 3])

In [24]:
dis = distances.unsqueeze(dim=0)
dis.shape

torch.Size([1, 40, 3])

In [26]:
out = torch.rand((1,40,18))
input = torch.cat([out, dis], dim=2)
input.shape

torch.Size([1, 40, 21])

In [27]:
for j in range(5):
    print(j)

0
1
2
3
4


In [38]:
def get_data_by_type(out, label, i):
    os, ls = [], []
    
    for j in range(3):
        o = out[:, 2*i + (6*j) : 2*(i+1) + (6*j)]
        os.append(o)
        l = label[:, 2*i + (6*j) : 2*(i+1) + (6*j)]
        ls.append(l)
    
    out = torch.cat(os, dim=1)
    label = torch.cat(ls, dim=1)
    
    return out, label

In [39]:
t = torch.rand((4, 18))
l = torch.rand((4, 18))
o_losses = []
t_losses = []

In [40]:
lf = MSELoss()

In [41]:
for i in range(3):
    # Object specific loss
    out = t[:, i*6 : (i+1)*6]
    label = l[:, i*6 : (i+1)*6]
    object_loss = lf(out, label)
    o_losses.append(object_loss.item())
    
    # Data specific loss (coords, orientation, force)
    out2, label2 = get_data_by_type(t, l, i)
    data_loss = lf(out2, label2)
    t_losses.append(data_loss.item())

In [42]:
o_losses, t_losses

([0.22520773112773895, 0.12353353947401047, 0.1602531373500824],
 [0.09188965708017349, 0.22642183303833008, 0.19068290293216705])

In [43]:
sum(o_losses), sum(t_losses)

(0.5089944079518318, 0.5089943930506706)

In [4]:
import itertools as it
import torch

In [5]:
x = torch.rand((3,3))
y = torch.rand((3,3))
z = torch.rand((3,3))

In [6]:
tensors = [x, y, z]
test = [[1,2], [3,4], [5,6]]

In [11]:
combs = it.combinations(tensors, 2)
combs = it.combinations(test, 2)

In [12]:
for com in combs:
    print(com)

([1, 2], [3, 4])
([1, 2], [5, 6])
([3, 4], [5, 6])


In [32]:
lr = [0.001, 0.0001]
wd = [0.0, 0.01]
loss = [torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.HuberLoss]
batchsize = [180, 360]
tf_steps = [80, 120]

params = {
    'lr': lr,
    'wd': wd,
    'loss': loss,
    #'batchsize': batchsize,
    #'tf_steps': tf_steps,
}
for param in params.items():
    print(param)

('lr', [0.001, 0.0001])
('wd', [0.0, 0.01])
('loss', [<class 'torch.nn.modules.loss.MSELoss'>, <class 'torch.nn.modules.loss.L1Loss'>, <class 'torch.nn.modules.loss.HuberLoss'>])


In [33]:
vals = params.values()
keys = params.keys()
vals, list(keys)

(dict_values([[0.001, 0.0001], [0.0, 0.01], [<class 'torch.nn.modules.loss.MSELoss'>, <class 'torch.nn.modules.loss.L1Loss'>, <class 'torch.nn.modules.loss.HuberLoss'>]]),
 ['lr', 'wd', 'loss'])

In [34]:
combinations = list(it.product(*vals))
combinations

[(0.001, 0.0, torch.nn.modules.loss.MSELoss),
 (0.001, 0.0, torch.nn.modules.loss.L1Loss),
 (0.001, 0.0, torch.nn.modules.loss.HuberLoss),
 (0.001, 0.01, torch.nn.modules.loss.MSELoss),
 (0.001, 0.01, torch.nn.modules.loss.L1Loss),
 (0.001, 0.01, torch.nn.modules.loss.HuberLoss),
 (0.0001, 0.0, torch.nn.modules.loss.MSELoss),
 (0.0001, 0.0, torch.nn.modules.loss.L1Loss),
 (0.0001, 0.0, torch.nn.modules.loss.HuberLoss),
 (0.0001, 0.01, torch.nn.modules.loss.MSELoss),
 (0.0001, 0.01, torch.nn.modules.loss.L1Loss),
 (0.0001, 0.01, torch.nn.modules.loss.HuberLoss)]

In [35]:
save_path = "data/model/tuning/"
for i, combinations in enumerate(combinations):

    print(i, lr, wd, loss)

0 0.001 0.0 <class 'torch.nn.modules.loss.MSELoss'>
1 0.001 0.0 <class 'torch.nn.modules.loss.L1Loss'>
2 0.001 0.0 <class 'torch.nn.modules.loss.HuberLoss'>
3 0.001 0.01 <class 'torch.nn.modules.loss.MSELoss'>
4 0.001 0.01 <class 'torch.nn.modules.loss.L1Loss'>
5 0.001 0.01 <class 'torch.nn.modules.loss.HuberLoss'>
6 0.0001 0.0 <class 'torch.nn.modules.loss.MSELoss'>
7 0.0001 0.0 <class 'torch.nn.modules.loss.L1Loss'>
8 0.0001 0.0 <class 'torch.nn.modules.loss.HuberLoss'>
9 0.0001 0.01 <class 'torch.nn.modules.loss.MSELoss'>
10 0.0001 0.01 <class 'torch.nn.modules.loss.L1Loss'>
11 0.0001 0.01 <class 'torch.nn.modules.loss.HuberLoss'>


In [36]:
l1 = [0,1,2]
l2 = [3,4,5]

In [37]:
for x, y in zip(l1, l2):
    print(x, y)

0 3
1 4
2 5


In [38]:
import numpy as np

In [40]:
mins = np.zeros(len(combinations))
mins

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [70]:
for i in range(len(combinations)):
    mins[i] = np.random.uniform()
mins, combinations

(array([0.17144111, 0.99726316, 0.17916047, 0.88041696, 0.26430295,
        0.753847  , 0.20012265, 0.55524662, 0.25723905, 0.93069162,
        0.70034084, 0.66409806]),
 array([[0.001, 0.0, <class 'torch.nn.modules.loss.MSELoss'>],
        [0.001, 0.0, <class 'torch.nn.modules.loss.L1Loss'>],
        [0.001, 0.0, <class 'torch.nn.modules.loss.HuberLoss'>],
        [0.001, 0.01, <class 'torch.nn.modules.loss.MSELoss'>],
        [0.001, 0.01, <class 'torch.nn.modules.loss.L1Loss'>],
        [0.001, 0.01, <class 'torch.nn.modules.loss.HuberLoss'>],
        [0.0001, 0.0, <class 'torch.nn.modules.loss.MSELoss'>],
        [0.0001, 0.0, <class 'torch.nn.modules.loss.L1Loss'>],
        [0.0001, 0.0, <class 'torch.nn.modules.loss.HuberLoss'>],
        [0.0001, 0.01, <class 'torch.nn.modules.loss.MSELoss'>],
        [0.0001, 0.01, <class 'torch.nn.modules.loss.L1Loss'>],
        [0.0001, 0.01, <class 'torch.nn.modules.loss.HuberLoss'>]],
       dtype=object))

In [71]:
idx = np.argpartition(mins, 5)
idx

array([ 0,  2,  6,  8,  4,  7, 11,  1,  3,  9, 10,  5], dtype=int64)

In [74]:
m = mins[idx[:5]]

In [75]:
combinations = np.array(combinations)
c = combinations[idx[:5]]

In [78]:
for (mm, cc) in zip(m, c):
    print(mm, cc.tolist())

0.17144111347618152 [0.001, 0.0, <class 'torch.nn.modules.loss.MSELoss'>]
0.1791604713424182 [0.001, 0.0, <class 'torch.nn.modules.loss.HuberLoss'>]
0.20012265090669645 [0.0001, 0.0, <class 'torch.nn.modules.loss.MSELoss'>]
0.2572390518697609 [0.0001, 0.0, <class 'torch.nn.modules.loss.HuberLoss'>]
0.2643029521778283 [0.001, 0.01, <class 'torch.nn.modules.loss.L1Loss'>]


In [79]:
idx[0]

0